<a href="https://colab.research.google.com/github/Plus-blank/2021-11-26_Machine-Learning_HousePrices/blob/main/%E6%88%BF%E5%83%B9%E9%A0%90%E6%B8%AC%E7%A8%8B%E5%BC%8F%E7%A2%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#參考網路教學，與同學分享的教學加上老師的上課講解

#數據讀取
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from keras.callbacks import *


In [2]:
#導入google drive
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [5]:
DATA_HOUSE = '/content/gdrive/MyDrive/house'

In [6]:
a_datas = pd.read_csv(f'{DATA_HOUSE}/train-v3.csv') #用pandas讀取csv檔案
a_datas #看一下數值

,id,price,sale_yr,sale_month,sale_day,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,5615100330,200000,2015,3,27,4,2.00,1900,8160,1,0,0,3,7,1900,0,1975,0,98022,47.2114,-121.986,1280,6532
1,8835900086,350000,2014,9,2,4,3.00,3380,16133,1,0,1,3,8,2330,1050,1959,0,98118,47.5501,-122.261,2500,11100
2,9510900270,254000,2014,12,11,3,2.00,2070,9000,1,0,0,4,7,1450,620,1969,0,98023,47.3085,-122.376,1630,7885
3,2621600015,175000,2015,4,30,3,1.00,1150,8924,1,0,0,3,6,1150,0,1943,0,98030,47.3865,-122.217,1492,8924
4,8078350090,619000,2015,3,31,3,2.50,2040,7503,2,0,0,3,8,2040,0,1987,0,98029,47.5718,-122.021,2170,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12962,9253900354,580000,2014,7,1,3,2.50,2200,11000,2,0,2,3,9,2200,0,1978,0,98008,47.5916,-122.112,2200,12851
12963,9510300130,598000,2014,6,28,4,2.50,3130,40918,2,0,0,3,9,3130,0,1994,0,98045,47.4761,-121.723,2760,35440
12964,1105000373,252500,2015,5,6,2,1.50,1110,986,2,0,0,3,7,950,160,2009,0,98118,47.5427,-122.272,1110,3515
12965,3629990280,497000,2014,6,23,3,2.25,1630,3817,2,0,0,3,7,1630,0,2005,0,98029,47.5485,-121.999,1630,3348


In [7]:
a_datas.dropna() #去空值看看，測試結果沒有空值

,id,price,sale_yr,sale_month,sale_day,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,5615100330,200000,2015,3,27,4,2.00,1900,8160,1,0,0,3,7,1900,0,1975,0,98022,47.2114,-121.986,1280,6532
1,8835900086,350000,2014,9,2,4,3.00,3380,16133,1,0,1,3,8,2330,1050,1959,0,98118,47.5501,-122.261,2500,11100
2,9510900270,254000,2014,12,11,3,2.00,2070,9000,1,0,0,4,7,1450,620,1969,0,98023,47.3085,-122.376,1630,7885
3,2621600015,175000,2015,4,30,3,1.00,1150,8924,1,0,0,3,6,1150,0,1943,0,98030,47.3865,-122.217,1492,8924
4,8078350090,619000,2015,3,31,3,2.50,2040,7503,2,0,0,3,8,2040,0,1987,0,98029,47.5718,-122.021,2170,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12962,9253900354,580000,2014,7,1,3,2.50,2200,11000,2,0,2,3,9,2200,0,1978,0,98008,47.5916,-122.112,2200,12851
12963,9510300130,598000,2014,6,28,4,2.50,3130,40918,2,0,0,3,9,3130,0,1994,0,98045,47.4761,-121.723,2760,35440
12964,1105000373,252500,2015,5,6,2,1.50,1110,986,2,0,0,3,7,950,160,2009,0,98118,47.5427,-122.272,1110,3515
12965,3629990280,497000,2014,6,23,3,2.25,1630,3817,2,0,0,3,7,1630,0,2005,0,98029,47.5485,-121.999,1630,3348


In [8]:
X_train = a_datas.drop(['price','id'],axis=1).values
Y_train = a_datas['price'].values

In [9]:
b_datas = pd.read_csv(f'{DATA_HOUSE}/valid-v3.csv')
X_valid = b_datas.drop(['price','id'],axis=1).values
Y_valid = b_datas['price'].values

In [10]:
c_datas = pd.read_csv(f'{DATA_HOUSE}/test-v3.csv')
X_test = c_datas.drop(['id'],axis=1).values

In [11]:
transfer = StandardScaler().fit(X_train) #正規化 還不是很理解意思
X_train = transfer.fit_transform(X_train)
X_valid_1 = transfer.fit_transform(X_valid)
X_test = transfer.fit_transform(X_test)

In [12]:
model = Sequential()
model.add(Dense(units=90, input_dim=X_train.shape[1], kernel_initializer='normal',activation='relu'))
model.add(Dense(units=150, kernel_initializer='normal',activation='relu')) #activation是輸出
model.add(Dense(units=300, kernel_initializer='normal',activation='relu'))
model.add(Dense(units=300, kernel_initializer='normal',activation='relu'))
model.add(Dense(units=200, kernel_initializer='normal',activation='relu'))
model.add(Dense(units=90, kernel_initializer='normal',activation='relu'))
model.add(Dense(units=1, kernel_initializer='normal', activation='relu'))


model.compile(loss='MAE', optimizer='adam',) #規則要loss=MAE
epochs = 110  #epochs調高batch_size就要調低_反之也是
batch_size = 39

In [25]:
file_name=str(epochs)+'_'+str(batch_size) #開始讓模型跑
TB=TensorBoard(log_dir='logs/'+file_name, histogram_freq=0)
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(X_valid_1, Y_valid),callbacks=[TB])

Epoch 1/110
333/333 [==============================] - 2s 6ms/step - loss: 51085.0664 - val_loss: 70124.3359
Epoch 2/110
333/333 [==============================] - 2s 6ms/step - loss: 51265.2305 - val_loss: 68484.1719
Epoch 3/110
333/333 [==============================] - 2s 6ms/step - loss: 50588.7109 - val_loss: 68961.7344
Epoch 4/110
333/333 [==============================] - 2s 6ms/step - loss: 51008.1367 - val_loss: 68447.0703
Epoch 5/110
333/333 [==============================] - 2s 6ms/step - loss: 50196.5508 - val_loss: 68839.9141
Epoch 6/110
333/333 [==============================] - 2s 6ms/step - loss: 50785.6836 - val_loss: 68576.7266
Epoch 7/110
333/333 [==============================] - 2s 6ms/step - loss: 50277.4570 - val_loss: 67500.2109
Epoch 8/110
333/333 [==============================] - 2s 6ms/step - loss: 49020.6250 - val_loss: 67750.9062
Epoch 9/110
333/333 [==============================] - 2s 6ms/step - loss: 49895.7383 - val_loss: 70832.4844
Epoch 10/110
333/33

In [26]:
model.save('h5/'+file_name+'.h5')

In [27]:
test_predict = model.predict(X_test)

In [28]:
with open("test.csv","w") as f :
  f.write("id,price\n")
  for i in range(len(test_predict)):
    f.write(str(i+1)+","+str(float(test_predict[i]))+ "\n")
